In [1]:
# Downloading preprocessed data, embeddings, models, and configs
!cd ~/experiments/knn4qa ; wget http://boytsov.info/datasets/knn4qa_demo2019-07-12.tar.bz2

--2019-07-12 02:56:31--  http://boytsov.info/datasets/knn4qa_demo2019-07-12.tar.bz2
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44206527 (42M) [application/x-bzip2]
Saving to: ‘knn4qa_demo2019-07-12.tar.bz2’

knn4qa_demo2019-07- 100%[===================>]  42.16M  2.59MB/s    in 11s     

2019-07-12 02:56:43 (3.73 MB/s) - ‘knn4qa_demo2019-07-12.tar.bz2’ saved [44206527/44206527]



In [2]:
# Unpack the data 
!cd ~/experiments/knn4qa ; tar jxvf knn4qa_demo2019-07-12.tar.bz2

x collections/squad/
x collections/squad/misc/
x collections/squad/embeddings/
x collections/squad/input_data/
x collections/squad/results/
x collections/squad/results/feat_exper/
x collections/squad/results/feat_exper/dev1/
x collections/squad/results/feat_exper/dev1/expers/
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/exper/
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/exper.log
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/rep/
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/trec_runs/
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/rep/out_250.gdeval.bz2
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/rep/out_250.tsv
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/rep/out_50.rep
x collections/squad/results/feat_exper/dev1/expers/bm25_lucene/rep/out_500.rep
x collections/squad/results/feat_exper/dev1/expers/b

In [3]:
# Create Lucene index
!cd ~/experiments/knn4qa ; scripts/index/create_lucene_index.sh squad

Data directory: collections/squad/input_data
Index directory: collections/squad/lucene_index
Using the data input file: AnswerFields.jsonl
MAVEN_OPTS=-Xms8388608k -Xmx14680064k -server
mvn -o compile exec:java -Dexec.mainClass=edu.cmu.lti.oaqa.knn4qa.apps.LuceneIndexer -Dexec.args='-input_data_dir collections/squad/input_data -index_dir collections/squad/lucene_index -data_sub_dirs bitext,dev1,dev2,test,train -data_file AnswerFields.jsonl' 
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 3.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ knn4qa ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[I

In [4]:
# Create forward index for two fields: text and text_unlemm
# One means that we keep word sequences
# The last argument is storage engine for the forward index
!cd ~/experiments/knn4qa ; scripts/index/create_fwd_index.sh squad "text text_unlemm" 0 "lucene"

Data directory: collections/squad/input_data
Index directory: collections/squad/forward_index/
Storing word id seq param: 
Index type:                lucene
Field list:                text text_unlemm
MAVEN_OPTS=-Xms8388608k -Xmx14680064k -server
mvn -o compile exec:java -Dexec.mainClass=edu.cmu.lti.oaqa.knn4qa.apps.BuildFwdIndexApp -Dexec.args='-fwd_index_type lucene -input_data_dir collections/squad/input_data -index_dir collections/squad/forward_index/ -data_sub_dirs bitext,dev1,dev2,test,train -data_file AnswerFields.jsonl -field_name text' 
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 3.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-res

In [5]:
# Export index to NMSLIB
!cd ~/experiments/ ; mkdir knn4qa/collections/squad/nmslib
!cd ~/experiments/knn4qa ; target/appassembler/bin/ExportToNMSLIBSparse \
-extr_json collections/squad/misc/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json \
-model_file  collections/squad/misc/out_squad_train_20.model  \
-fwd_index_dir collections/squad/forward_index \
-embed_dir collections/squad/embeddings \
-out_file collections/squad/nmslib/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin


MAVEN_OPTS=-Xms11184810k -Xmx14680064k -server
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< edu.cmu.lti.oaqa:knn4qa >-----------------------
[INFO] Building Exploring if k-NN search can be useful for non-factoid QA. 3.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[WARNING] The POM for umass:RankLib:jar:2.7 is missing, no dependency information available
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ knn4qa ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[INFO] 
[INFO] --- maven-compiler-plugin:2.3.2:compile (default-compile) @ knn4qa ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- exec-maven-plugin:1.3.2:java (default-cli) @ knn4qa ---
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBi

In [6]:
# Finally starting the query server on port 8080 and creating the index using HNSW
!cd ~/experiments/nmslib/query_server/cpp_client_server ; \
    ./query_server -i ~/experiments/knn4qa/collections/squad/nmslib/cosine_fusion_dim=100_squad_emb_unlemm_epoch1.json_sparse.bin \
    -s cosinesimil_sparse_bin_fast -m hnsw -c M=20,efConstruction=200 -t ef=200 -p 8080 

2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: INT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: FLOAT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: dummy distance type: DOUBLE
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: bit_hamming distance type: INT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: leven distance type: INT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: normleven distance type: FLOAT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: kldivfast distance type: FLOAT
2019-07-12 02:57:21 spacefactory.h:42 (Register) [INFO] Registering at the factory, space: kldivfast distance type: DOUBLE
2019-07-12 02:57:21 spacefactory.h:42 (Reg

2019-07-12 02:57:21 QueryService_server.cpp:125 (QueryServiceHandler) [INFO] Creating a new index copy
2019-07-12 02:57:21 hnsw.cc:180 (CreateIndex) [INFO] M                   = 20
2019-07-12 02:57:21 hnsw.cc:181 (CreateIndex) [INFO] indexThreadQty      = 8
2019-07-12 02:57:21 hnsw.cc:182 (CreateIndex) [INFO] efConstruction      = 200
2019-07-12 02:57:21 hnsw.cc:183 (CreateIndex) [INFO] maxM			          = 20
2019-07-12 02:57:21 hnsw.cc:184 (CreateIndex) [INFO] maxM0			          = 40
2019-07-12 02:57:21 hnsw.cc:186 (CreateIndex) [INFO] mult                = 0.333808
2019-07-12 02:57:21 hnsw.cc:187 (CreateIndex) [INFO] skip_optimized_index= 0
2019-07-12 02:57:21 hnsw.cc:188 (CreateIndex) [INFO] delaunay_type       = 2
2019-07-12 02:57:21 hnsw.cc:470 (SetQueryTimeParams) [INFO] Set HNSW query-time parameters:
2019-07-12 02:57:21 hnsw.cc:471 (SetQueryTimeParams) [INFO] ef(Search)         =20
2019-07-12 02:57:21 hnsw.cc:472 (SetQueryTimeParams) [INFO] algoType           =2

0%   10   20   3